# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [81]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [82]:
# Read CSV file created in the weatherPy exercise:
cities = "../WeatherPy/output_data/cities.csv"
cities_df = pd.read_csv(cities, encoding="ISO-8859-1")
cities_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yefira,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,rikitea,-23.1203,-134.9692,22.11,74.0,14.0,8.45,PF,1.662781e+09
2,2,touros,-5.1989,-35.4608,23.11,84.0,0.0,5.76,BR,1.662781e+09
3,3,narsaq,60.9167,-46.0500,8.12,62.0,100.0,1.98,GL,1.662781e+09
4,4,chokurdakh,70.6333,147.9167,5.16,87.0,100.0,4.60,RU,1.662781e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [83]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [84]:
# HINT: be sure to handle NaN values
cities_df = cities_df.dropna()

# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
cities_df = cities_df.dropna()
humidity = cities_df["Humidity"].astype(float)

In [85]:
# Find maximum humidity value to be plugged into the heat_layer step below:
humidity_max = cities_df["Humidity"].max()
humidity_max

100.0

In [87]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=humidity_max,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
# Removing rows with Max Temp >= 27
max_temperature_df = cities_df[cities_df["Max Temp"] < 27] 

In [22]:
# Removing rows with Max Temp <= 21
min_temperature_df = max_temperature_df[max_temperature_df["Max Temp"] > 21] 


In [23]:
# Removing rows with Wind Speed > 4.5:
wind_speed_df = min_temperature_df[min_temperature_df["Wind Speed"] >= 4.5] 

In [24]:
# Removing rows with Cloudiness not equal to 0:
perfect_weather_df = wind_speed_df[wind_speed_df["Cloudiness"] == 0 ]
indexed_perfect_weather_df = perfect_weather_df.reset_index()

In [25]:
# Remove index and Unnamed: 0 coloumns:
del indexed_perfect_weather_df["index"]
del indexed_perfect_weather_df["Unnamed: 0"]


In [26]:
# Print perfect weather:
indexed_perfect_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,touros,-5.1989,-35.4608,23.11,84.0,0.0,5.76,BR,1.662781e+09
1,methoni,36.8197,21.7049,22.90,85.0,0.0,5.51,GR,1.662781e+09
2,los llanos de aridane,28.6585,-17.9182,21.86,96.0,0.0,4.63,ES,1.662781e+09
3,astoria,46.1879,-123.8313,23.73,46.0,0.0,6.69,US,1.662781e+09
4,karratha,-20.7377,116.8463,26.06,33.0,0.0,4.88,AU,1.662781e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
# Add a new coloumn " Nearst Holte" to indexed_perfect_weather_df and print the data frame:
indexed_perfect_weather_df["Nearest Hotel"] = ""
indexed_perfect_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearest Hotel
0,touros,-5.1989,-35.4608,23.11,84.0,0.0,5.76,BR,1.662781e+09,
1,methoni,36.8197,21.7049,22.90,85.0,0.0,5.51,GR,1.662781e+09,
2,los llanos de aridane,28.6585,-17.9182,21.86,96.0,0.0,4.63,ES,1.662781e+09,
3,astoria,46.1879,-123.8313,23.73,46.0,0.0,6.69,US,1.662781e+09,
4,karratha,-20.7377,116.8463,26.06,33.0,0.0,4.88,AU,1.662781e+09,
5,abnub,27.2696,31.1511,24.12,59.0,0.0,4.83,EG,1.662781e+09,
6,vila velha,-20.3297,-40.2925,22.97,76.0,0.0,5.66,BR,1.662781e+09,


In [88]:

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}


# Use the lat/lng we recovered to identify hotels
for index, row in indexed_perfect_weather_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_df = requests.get(base_url, params=params)
    

    # convert to json
    hotel_df = hotel_df.json()


    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        indexed_perfect_weather_df.loc[index, "Nearest Hotel"] = hotel_df["results"][1]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [64]:
indexed_perfect_weather_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearest Hotel
0,touros,-5.1989,-35.4608,23.11,84.0,0.0,5.76,BR,1.662781e+09,INN NEW HORIZON
1,methoni,36.8197,21.7049,22.90,85.0,0.0,5.51,GR,1.662781e+09,Methoni Beach Hotel
2,los llanos de aridane,28.6585,-17.9182,21.86,96.0,0.0,4.63,ES,1.662781e+09,Valle Aridane
3,astoria,46.1879,-123.8313,23.73,46.0,0.0,6.69,US,1.662781e+09,Best Western Astoria Bayfront Hotel
4,karratha,-20.7377,116.8463,26.06,33.0,0.0,4.88,AU,1.662781e+09,ibis Styles Karratha
5,abnub,27.2696,31.1511,24.12,59.0,0.0,4.83,EG,1.662781e+09,Post Office - Abnoub
6,vila velha,-20.3297,-40.2925,22.97,76.0,0.0,5.66,BR,1.662781e+09,Hotel Golden Tulip Porto Vitória


In [67]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in indexed_perfect_weather_df.iterrows()]
locations = indexed_perfect_weather_df[["Lat", "Lng"]]

In [80]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))